In [14]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

### Loading the dataset

In [15]:
movie_reviews_df = spark.read.csv("IMDB Dataset.csv", header=True, inferSchema=True)

### Preprocessing

### Naive Bayse implementation